In [1]:
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import arff
import data_processing as dp

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
data = arff.loadarff("../data/3year.arff")
df = pd.DataFrame(data[0])
dp.df_null_corr_process(df)

In [ ]:
X, y = get_Xy(df)
